In [1]:
from Bio.PDB import *
import urllib.request
import numpy as np
import pandas as pd
from math import sqrt
import time

In [2]:
cd C:\Users\Administrator\Desktop\temp

C:\Users\Administrator\Desktop\temp


In [9]:
peptidasesList = pd.read_csv("Y:/Yuan/temp/MCSA_EC3.4_peptidases.csv")
peptidasesList = peptidasesList[peptidasesList.iloc[:,4] == "residue"]

In [10]:
peptidasesList = peptidasesList.reset_index(drop=True)

In [11]:
peptidasesList[0:5]

,M-CSA ID,Uniprot IDs,PDB,EC,residue/reactant/product/cofactor,PDB code,chain/kegg compound,resid/chebi id,function location/name,role,role type,role group
0,M0587,P00727,1lam,3.4.11.1,residue,Lys,A,262.0,side_chain,electrostatic stabiliser,spectator,electrostatic interaction
1,M0587,P00727,1lam,3.4.11.1,residue,Arg,A,336.0,side_chain,electrostatic stabiliser,spectator,electrostatic interaction
2,M0167,Q01693,1lok,3.4.11.10,residue,His,A,97.0,side_chain,metal ligand,interaction,None
3,M0167,Q01693,1lok,3.4.11.10,residue,His,A,97.0,side_chain,metal ligand,interaction,None
4,M0167,Q01693,1lok,3.4.11.10,residue,His,A,97.0,side_chain,metal ligand,interaction,None


In [12]:
bindingSiteDic = {}
for i in range(len(peptidasesList)):
    #print(bindingSiteDic)
    if peptidasesList.loc[i,"PDB"] not in bindingSiteDic:
        bindingSiteDic[peptidasesList.loc[i,"PDB"]] = {peptidasesList.loc[i,"chain/kegg compound"]: [peptidasesList.loc[i,"resid/chebi id"]]}
    elif peptidasesList.loc[i,"chain/kegg compound"] not in bindingSiteDic[peptidasesList.loc[i,"PDB"]]:
         bindingSiteDic[peptidasesList.loc[i,"PDB"]] = {peptidasesList.loc[i,"chain/kegg compound"]: [peptidasesList.loc[i,"resid/chebi id"]]}
    else:
        bindingSiteDic[peptidasesList.loc[i,"PDB"]][peptidasesList.loc[i,"chain/kegg compound"]].append(peptidasesList.loc[i,"resid/chebi id"])
for protein in bindingSiteDic:
    for chain in bindingSiteDic[protein]:
        bindingSiteDic[protein][chain] = list(set(bindingSiteDic[protein][chain])) 

In [13]:
uniqueList = peptidasesList[["PDB","chain/kegg compound"]].drop_duplicates()

In [15]:
uniqueList.reset_index(drop = True).iloc[0:10,]

,PDB,chain/kegg compound
0,1lam,A
1,1lok,A
2,1xgm,A
3,1b65,A
4,1ei5,A
5,1azw,A
6,1a16,A
7,1itq,A
8,1fy2,A
9,1r44,A


In [16]:
backbone = ["N","CA","C","O"]
aminoAcidCodes = ["ALA","ARG","ASN","ASP","CYS","GLN","GLY","GLU","HIS","ILE","LEU","LYS",
                 "MET","PHE","PRO","PYL","SER","SEC","THR","TRP","TYR","TRP","VAL"]

In [17]:
pdbID = uniqueList.iloc[34,0]
chainOrder = uniqueList.iloc[34,1]
PDB = PDBList()
PDB.retrieve_pdb_file(pdb_code = pdbID, pdir = "Y:/Yuan/temp", file_format="pdb")

'Y:/Yuan/temp\\pdb1ds2.ent'

In [18]:
p = PDBParser()
structure = p.get_structure("X","Y:/Yuan/temp/pdb"+pdbID+".ent")

C:\Users\Administrator\Anaconda3\envs\pdbPy3\lib\site-packages\Bio\PDB\StructureBuilder.py:91: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 2036.
  PDBConstructionWarning)
C:\Users\Administrator\Anaconda3\envs\pdbPy3\lib\site-packages\Bio\PDB\StructureBuilder.py:91: PDBConstructionWarning: WARNING: Chain I is discontinuous at line 2148.
  PDBConstructionWarning)


In [19]:
oneChain = pd.DataFrame(columns = ["Seq","Residue","Center","Direction"])
if structure.header["resolution"]<=3.0:
    if chainOrder in [x.id for x in list(structure[0].get_chains())]: #Chain information not in pdb file
        for residue in structure[0][chainOrder]: 
            if residue.get_resname() in aminoAcidCodes: # Only treat common amino acid
                if len(list(residue.get_atoms())) >3:
                    if residue.get_resname() != "GLY":  # Glysine as a special case
                        point = vectors.Vector([0,0,0])
                        for atom in residue:
                            if(atom.get_name() not in backbone):
                                point = point + atom.get_vector()
                        center = point.__div__(len(residue)-4)
                        cToRGroup = residue["CA"].get_vector()-center
                        oneChain.loc[len(oneChain)] = [residue.get_id()[1],residue.get_resname(),center,cToRGroup]        
                    else:
                        center = residue["CA"].get_vector()
                        cToRGroup = center - (residue["C"].get_vector()+residue["N"].get_vector()+residue["O"].get_vector()).__div__(3)
                        oneChain.loc[len(oneChain)] = [residue.get_id()[1],residue.get_resname(),center,cToRGroup]                   

In [20]:
distanceMatrix = pd.DataFrame(columns = list(oneChain.iloc[:,0]),index = list(oneChain.iloc[:,0]))

In [21]:
print(time.time())
numResidue = len(oneChain)
for row in range(0,numResidue):
    if row % 50 ==0:
        print(str(row)+"th row")
    for column in range(0,numResidue):
        coordinatesSubstraction = list(oneChain.loc[row,"Center"] - oneChain.loc[column,"Center"])
        distanceMatrix.iloc[row,column] = sqrt(sum(list(map(lambda x:x*x, coordinatesSubstraction))))
print(time.time())

1562533659.0931923
1562533659.0941727


In [ ]:
for eachRow in range(0,len(uniqueList)):
    pdbID = uniqueList.iloc[eachRow,0]
    chainOrder = uniqueList.iloc[eachRow,1]
    PDB = PDBList()
    PDB.retrieve_pdb_file(pdb_code = pdbID, pdir = "Y:/Yuan/temp", file_format="pdb")
    p = PDBParser()
    structure = p.get_structure("X","Y:/Yuan/temp/pdb"+pdbID+".ent")
    oneChain = pd.DataFrame(columns = ["Seq","Residue","Center","Direction"])
    if structure.header["resolution"]<=3.0:
        if chainOrder in [x.id for x in list(structure[0].get_chains())]:
            for residue in structure[0][chainOrder]:
                if residue.get_resname() in aminoAcidCodes:
                    if len(list(residue.get_atoms())) >3:
                        if residue.get_resname() != "GLY":
                            point = vectors.Vector([0,0,0])
                            for atom in residue:
                                if(atom.get_name() not in backbone):
                                    point = point + atom.get_vector()
                            center = point.__div__(len(residue)-4)
                            cToRGroup = residue["CA"].get_vector()-center
                            oneChain.loc[len(oneChain)] = [residue.get_id()[1],residue.get_resname(),center,cToRGroup]        
                        else:
                            center = residue["CA"].get_vector()
                            cToRGroup = center - (residue["C"].get_vector()+residue["N"].get_vector()+residue["O"].get_vector()).__div__(3)
                            oneChain.loc[len(oneChain)] = [residue.get_id()[1],residue.get_resname(),center,cToRGroup]    
            distanceMatrix = pd.DataFrame(columns = list(oneChain.iloc[:,0]),index = list(oneChain.iloc[:,0]))
            print(time.time())
            numResidue = len(oneChain)
            for row in range(0,numResidue):
                if row % 50 ==0:
                    print(str(row)+"th row")
                for column in range(0,numResidue):
                    coordinatesSubstraction = list(oneChain.loc[row,"Center"] - oneChain.loc[column,"Center"])
                    distanceMatrix.iloc[row,column] = sqrt(sum(list(map(lambda x:x*x, coordinatesSubstraction))))
        print(time.time())

## The below is going to verify several residues for calculation correctness.

In [184]:
print(structure[0]["E"][2])
print(list(structure[0]["E"][2].get_atoms()))
for atom in structure[0]["E"][2]:
    print(atom.get_name()+": "+ str(atom.get_vector()))

<Residue SER het=  resseq=2 icode= >
[<Atom N>, <Atom CA>, <Atom C>, <Atom O>, <Atom CB>, <Atom OG>]
N: <Vector -29.51, 126.85, 122.03>
CA: <Vector -29.91, 128.21, 121.70>
C: <Vector -31.43, 128.27, 121.58>
O: <Vector -32.08, 127.26, 121.33>
CB: <Vector -29.27, 128.65, 120.38>
OG: <Vector -29.75, 129.93, 119.98>


In [185]:
(structure[0]["E"][2]["CB"].get_vector() + structure[0]["E"][2]["OG"].get_vector()).__div__(2)

<Vector -29.51, 129.29, 120.18>

In [191]:
structure[0]["E"][2]["CA"].get_vector() - (structure[0]["E"][2]["CB"].get_vector() + structure[0]["E"][2]["OG"].get_vector()).__div__(2) 

<Vector -0.40, -1.08, 1.52>

In [192]:
distanceMatrix

,1,2,3,4,5,6,7,8,9,10,...,434,435,436,437,438,439,440,441,442,443
1,0,8.14334,9.49318,11.9676,16.5674,16.608,22.3763,16.2933,13.7233,20.2866,...,52.9758,53.5734,51.1488,52.5593,51.1303,47.143,47.9113,48.9832,46.9483,43.4618
2,8.14334,0,6.59657,3.91996,9.29266,9.94028,14.702,8.48322,6.33903,12.7622,...,48.1393,49.5998,46.8647,49.3297,47.5245,43.1027,44.4277,46.3585,43.7527,39.5732
3,9.49318,6.59657,0,8.24839,9.89221,12.5605,16.5215,9.56934,11.751,17.3311,...,54.1693,55.8794,53.1978,55.8113,53.8053,49.4248,50.9298,52.8441,50.0202,45.8219
4,11.9676,3.91996,8.24839,0,7.30787,8.8277,11.9364,5.37574,4.68122,10.0248,...,46.0038,47.9093,45.1914,48.099,45.9834,41.4673,43.2052,45.4014,42.4077,37.9617
5,16.5674,9.29266,9.89221,7.30787,0,5.0803,6.67015,3.7123,7.75575,9.77741,...,48.9483,51.4512,47.9391,51.8217,50.0154,44.9868,46.4911,49.7477,46.9417,42.025
6,16.608,9.94028,12.5605,8.8277,5.0803,0,6.8899,7.74761,6.40215,7.89684,...,47.0098,49.2213,45.2002,49.1175,47.9274,42.6539,43.5126,47.1699,44.9862,40.0454
7,22.3763,14.702,16.5215,11.9364,6.67015,6.8899,0,8.11085,10.3368,7.44384,...,45.4719,48.5419,44.5261,49.2772,47.5904,42.2108,43.7546,47.8848,45.0939,39.7763
8,16.2933,8.48322,9.56934,5.37574,3.7123,7.74761,8.11085,0,7.34814,9.599,...,47.4265,49.971,46.9495,50.6147,48.3699,43.5989,45.5641,48.3829,45.1596,40.3449
9,13.7233,6.33903,11.751,4.68122,7.75575,6.40215,10.3368,7.34814,0,6.57278,...,43.3671,45.2161,41.939,45.1366,43.5226,38.6617,39.9237,42.7068,40.1854,35.5436
10,20.2866,12.7622,17.3311,10.0248,9.77741,7.89684,7.44384,9.599,6.57278,0,...,39.458,42.0055,38.2376,42.4362,40.8132,35.544,37.0104,40.7448,38.0853,32.9226


# The below is catalytic traid 

In [3]:
PDB = PDBList()
PDB.retrieve_pdb_file(pdb_code = "2GMT", pdir = "Y:/Yuan/temp", file_format="pdb")

'Y:/Yuan/temp\\pdb2gmt.ent'

In [4]:
p = PDBParser()
structure = p.get_structure("X","Y:/Yuan/temp/pdb"+"2GMT"+".ent")

C:\Users\Administrator\Anaconda3\envs\pdbPy3\lib\site-packages\Bio\PDB\StructureBuilder.py:91: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 2221.
  PDBConstructionWarning)
C:\Users\Administrator\Anaconda3\envs\pdbPy3\lib\site-packages\Bio\PDB\StructureBuilder.py:91: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2242.
  PDBConstructionWarning)
C:\Users\Administrator\Anaconda3\envs\pdbPy3\lib\site-packages\Bio\PDB\StructureBuilder.py:91: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 2254.
  PDBConstructionWarning)
C:\Users\Administrator\Anaconda3\envs\pdbPy3\lib\site-packages\Bio\PDB\StructureBuilder.py:91: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 2389.
  PDBConstructionWarning)


In [5]:
backbone = ["N","CA","C","O"]
aminoAcidCodes = ["ALA","ARG","ASN","ASP","CYS","GLN","GLY","GLU","HIS","ILE","LEU","LYS",
                 "MET","PHE","PRO","PYL","SER","SEC","THR","TRP","TYR","VAL"]

In [18]:
oneChain = pd.DataFrame(columns = ["Seq","Residue","Center","Direction"])
chainOrder = "B"
if structure.header["resolution"]<=3.0:
    if chainOrder in [x.id for x in list(structure[0].get_chains())]: #Chain information not in pdb file
        for residue in structure[0][chainOrder]: 
            if residue.get_resname() in aminoAcidCodes: # Only treat common amino acid
                if len(list(residue.get_atoms())) >3:
                    if residue.get_resname() != "GLY":  # Glysine as a special case
                        point = vectors.Vector([0,0,0])
                        for atom in residue:
                            if(atom.get_name() not in backbone):
                                point = point + atom.get_vector()
                        center = point.__div__(len(residue)-4)
                        cToRGroup = residue["CA"].get_vector()-center
                        oneChain.loc[len(oneChain)] = [residue.get_id()[1],residue.get_resname(),center,cToRGroup]        
                    else:
                        center = residue["CA"].get_vector()
                        cToRGroup = center - (residue["C"].get_vector()+residue["N"].get_vector()+residue["O"].get_vector()).__div__(3)
                        oneChain.loc[len(oneChain)] = [residue.get_id()[1],residue.get_resname(),center,cToRGroup]

In [27]:
distanceMatrix = pd.DataFrame(columns = list(oneChain.iloc[:,0]),index = list(oneChain.iloc[:,0]))
angleMatrix = pd.DataFrame(columns = list(oneChain.iloc[:,0]),index = list(oneChain.iloc[:,0]))
numResidue = len(oneChain)
for row in range(0,numResidue):
    if row % 50 ==0:
        print(str(row)+"th row")
    for column in range(0,numResidue):
        coordinatesSubstraction = list(oneChain.loc[row,"Center"] - oneChain.loc[column,"Center"])
        distanceMatrix.iloc[row,column] = sqrt(sum(list(map(lambda x:x*x, coordinatesSubstraction))))
        v1 = oneChain.loc[row,"Direction"] 
        v2 = oneChain.loc[column,"Direction"]
        angleMatrix.iloc[row,column] = 180*np.arccos(np.dot(v1,v2)/(np.linalg.norm(v1) * np.linalg.norm(v2)))/np.pi
        

0th row


C:\Users\Administrator\Anaconda3\envs\pdbPy3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in arccos
  if sys.path[0] == '':


50th row
100th row


In [13]:
v1 = [1, -1, 0]
v2 = [1, 1, 0]
180*np.arccos(np.dot(v1,v2)/(np.linalg.norm(v1) * np.linalg.norm(v2)))/np.pi

90.0

In [25]:
oneChain.loc[1,]

Seq                                    17
Residue                               VAL
Center       <Vector 35.35, 85.85, 95.53>
Direction      <Vector 0.87, 1.22, -1.21>
Name: 1, dtype: object

In [31]:
angleMatrix.to_csv("angle.csv")
distanceMatrix.to_csv("distance.csv")
oneChain.to_csv("2GMT_info.csv")

['Seq', 'Residue', 'Center', 'Direction']